In [2]:
%%writefile ../src/OrderEvent.sol
pragma solidity ^0.6.0;

contract OrderEvent {
    uint unitPrice = 10;
    
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Overwriting ../src/OrderEvent.sol


In [3]:
!solc ../src/OrderEvent.sol --combined-json abi > ..\src\OrderEventABI.json

In [4]:
!solc ../src/OrderEvent.sol --combined-json bin > ..\src\OrderEventBIN.json

## 배포

In [49]:
%%writefile ../src/OrderEventDeploy.js
var Web3=require('web3');
var _abiJson = require('./OrderEventABI.json');
var _binJson = require('./OrderEventBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/OrderEventDeploy.js


In [50]:
!node ../src/OrderEventDeploy.js

- contract name:  ../src/OrderEvent.sol:OrderEvent
Deploying the contract from 0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf
hash: 0x2f75b791465c0342ff380daf574ecb84eedb93638994ae9f9da6b27969522549
---> The contract deployed to: 0xa1B8B096b473103c709483905E28914fCf48748b


## 사용

In [51]:
%%writefile ../src/OrderEventUse.js
var Web3=require('web3');
var fs=require('fs');
var _abiJson = require('./OrderEventABI.json');
var _binJson = require('./OrderEventBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _order = new web3.eth.Contract(_abiArray, '0xa1B8B096b473103c709483905E28914fCf48748b');

var event = _order.events.OrderLog({filter: {_from: web3.eth.accounts[0], _value: 30}, fromBlock: 0}, function (error, result) {
    if (!error) {
        var log = JSON.stringify(result.returnValues)
        console.log("Event fired: " + log);
        
        fs.writeFile("../src/OrderEventLog.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file");
            }
        });
        
        //대기 후 종료
        setTimeout(function() {
          process.exit(1);
        }, 1000);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);

    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    // this will fire an event
    my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    
    // this will fire another event
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    
    // this will NOT fire another event
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting ../src/OrderEventUse.js


In [52]:
!node ../src/OrderEventUse.js

- contract name:  ../src/OrderEvent.sol:OrderEvent
Account: 0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf
Balance before: 99999815993000000000
Event fired: {"0":"0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf","1":"0x1234","2":"30","_from":"0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf","_itemId":"0x1234","_value":"30"}
>> Writing to file
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 99998198572999999830
Balance diff: 1617420000002048


In [53]:
!type ..\src\OrderEventLog.txt

{"0":"0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf","1":"0x1234","2":"30","_from":"0xb8b7BCf69088077b8EE587E481A3F862fAb4E1Bf","_itemId":"0x1234","_value":"30"}
